<a href="https://colab.research.google.com/github/wootaehyeon/pytorch-Tutorial/blob/main/simple_NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
      super().__init__()
      self.fc1 = nn.Linear(in_features, h1)
      self.fc2 = nn.Linear(h1, h2)
      self.out = nn.Linear(h2, out_features)

    def forward(self,x):
      x = F.relu(self.fc1(x))
      x = F.relu(self.fc2(x))
      x = self.out(x)
      return x


In [37]:
torch.manual_seed(41)
model = Model()

In [38]:
url = 'https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv'
my_df = pd.read_csv(url)

In [39]:
my_df['species'] = my_df['species'].replace('setosa', 0.0)
my_df['species'] = my_df['species'].replace('versicolor', 1.0)
my_df['species'] = my_df['species'].replace('virginica', 2.0)
#change last

<ipython-input-39-3a4c28f35560>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  my_df['species'] = my_df['species'].replace('virginica', 2.0)


In [40]:
X = my_df.drop('species', axis=1).to_numpy().astype(np.float32)
y = my_df['species'].to_numpy().astype(np.int64)

In [41]:
X = X.values
y = y.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [ ]:
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [ ]:
#set the criterion of model to measure the error
criterion = nn.CrossEntropyLoss()
#Choose Adam Optimizer, lr = learning rate(iff error doesn't go down after)
optimizer = torch.optim.Adam(model.parameters(),lr = 0.01)

In [ ]:
#train our model
#Epoch?v(one run thru all the training data in our network)
epochs = 100 #100번 반복
losses = []
for i in range(epochs):
  #Go forward and get a prediction
  y_pred = model.forward(X_train) #Get predicted results

  #Measure the loss/error, gonna
  lose = criterion(y_pred,y_train) #predicted values vs the y_train

  #keep Track of our losses
  losses.append(loss.detach().numpy())

  #print every 10 epoch
  if i%10 == 0:
    print(f'Epoch : {i} and loss: {loss}')

  #Do some back propagation: take error rate of forward propagation and feed it back
  #thru the network to fine tune the weights
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

In [ ]:
plt.plot(range(epochs),losses)
plt.ylabel('Loss')
plt.xlabel('Epoch')

In [ ]:
#Evalutae Model on Test Data set
with torch.no_grad():  #Basically turn off back propagation
  y_eval = model.forward(X_test) #X_test are features from our test set,
  loss = criterion(y_eval,y_test) #find the loss or error

In [ ]:
correct = 0
with torch.no_grad():
  for i , data in enumerate(X_test):
    y_val = model.forward(data)

    print(f'{i+1}.) {str(y_val)}\t{y_test[i]}')

In [ ]:
if y_val.argmax().item() == y_test[i]:
  correct += 1
print(f'{correct} out of {len(y_test)} = {100*correct/len(y_test)}% correct')

In [47]:
new_iris = torch.tensor([4.7,3.2,1.3,0.2])

In [48]:
with torch.no_grad():
  print(model(new_iris))

tensor([0.1756, 0.2740, 0.2870])


In [49]:
#Save our NN model
torch.save(model.state_dict(),'my_really_awesome_iris_model.pt')



In [51]:
#Load the saved Model
new_model = Model()
new_model.load_state_dict(torch.load('my_really_awesome_iris_model.pt'))


<ipython-input-51-9edd1282ac8c>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  new_model.load_state_dict(torch.load('my_really_awesome_iris_model.pt'))


<All keys matched successfully>

In [52]:
#Make sure it loaded correctly
new_model.eval()


Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)